In [1]:
import pandas as pd
import os
import random
from sklearn.tree import DecisionTreeClassifier
import re

# Optional imports for analytics (gracefully handle if not available)
try:
    import matplotlib.pyplot as plt
    import seaborn as sns
    ANALYTICS_AVAILABLE = True
except ImportError:
    ANALYTICS_AVAILABLE = False
    print("Note: matplotlib/seaborn not available. Analytics visualizations will be disabled.")

# ------------------------------
# Configuration
# ------------------------------
FILE_NAME = "students.csv"
MAX_STUDENTS = 100
MIN_AGE = 18
MAX_AGE = 100
YEAR = 2026

# ------------------------------
# Programs & Descriptions
# ------------------------------
programs = {
    "Data Science": "Learn data analysis, visualization, and machine learning using Python.",
    "Backend Development": "Build server-side applications and APIs.",
    "Frontend Development": "Create responsive and interactive websites.",
    "Cybersecurity": "Learn how to protect systems and networks.",
    "UI/UX Design": "Design user-friendly digital experiences.",
    "Artificial Intelligence": "Understand intelligent systems and AI models.",
    "Cloud Computing": "Learn cloud infrastructure and deployment.",
    "Software Development": "Learn the full software development lifecycle.",
    "Data Analytics": "Analyze data to support business decisions.",
    "Product Management": "Plan and manage digital products.",
    "Product Design": "Design digital products with user focus.",
    "AI and Automation": "Build automated intelligent systems.",
    "IT Support": "Provide technical support and troubleshooting.",
    "Cloud DevOps": "Automate deployment and cloud operations.",
    "Virtual Assistant": "Learn remote administrative and productivity skills."
}

program_codes = {p: p[:2].upper() for p in programs}  # For registration IDs

# ------------------------------
# Program durations in weeks
# ------------------------------
program_durations = {
    "Data Science": 8,
    "Backend Development": 6,
    "Frontend Development": 6,
    "Cybersecurity": 8,
    "UI/UX Design": 4,
    "Artificial Intelligence": 10,
    "Cloud Computing": 6,
    "Software Development": 12,
    "Data Analytics": 6,
    "Product Management": 6,
    "Product Design": 4,
    "AI and Automation": 8,
    "IT Support": 4,
    "Cloud DevOps": 6,
    "Virtual Assistant": 4
}

# ------------------------------
# Helper Functions
# ------------------------------
def validate_email(email):
    """Validate email format"""
    pattern = r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'
    return re.match(pattern, email) is not None

def normalize_yes_no(response):
    """Normalize yes/no responses"""
    response = response.strip().lower()
    if response in ['yes', 'y']:
        return 'yes'
    elif response in ['no', 'n']:
        return 'no'
    return None

def is_duplicate(username, email=None):
    """Check if username or email already exists"""
    for s in students:
        if s["user_name"].lower() == username.lower():
            return True
        if email and s["user_email"].lower() == email.lower():
            return True
    return False

def show_programs():
    """Display all available programs"""
    print("\nAvailable Programs:\n")
    for i, (program, desc) in enumerate(programs.items(), start=1):
        print(f"{i}. {program}")
        print(f"   → {desc} (Duration: {program_durations[program]} weeks)\n")

def show_program_statistics():
    """Display registration statistics by program"""
    if not students:
        print("\nNo students registered yet.")
        return
    
    try:
        df_stats = pd.DataFrame(students)
        counts = df_stats["program"].value_counts()
        print("\n--- Program Registration Statistics ---")
        for program, count in counts.items():
            print(f"{program}: {count} student(s)")
        print(f"\nTotal registered students: {len(students)}/{MAX_STUDENTS}")
    except Exception as e:
        print(f"Error displaying statistics: {e}")

def search_by_registration_id():
    """Search and display student details by registration ID"""
    if not students:
        print("\nNo registrations found.")
        return
    
    reg_id = input("Enter your registration ID: ").strip()
    for student in students:
        if student["registration_id"] == reg_id:
            print("\n====================================")
            print(" REGISTRATION DETAILS ")
            print("====================================")
            print(f"Institute: {student['learning_institute']}")
            print(f"Name: {student['user_name']}")
            print(f"Program: {student['program']}")
            print(f"Email: {student['user_email']}")
            print(f"Age: {student['user_age']}")
            print(f"Registration ID: {student['registration_id']}")
            commitment_text = "20+ hours/week" if student.get("commitment") == "yes" else "Less than 20 hours/week"
            print(f"Commitment: {commitment_text}")
            print(f"Duration: {program_durations[student['program']]} weeks")
            print("====================================")
            return
    print("No record found with that registration ID.")

def login():
    """Simple login by email verification"""
    if not students:
        print("\nNo registered users yet. Please register first.")
        return
    
    email = input("Enter your registered email: ").strip().lower()
    user = next((s for s in students if s['user_email'].lower() == email), None)
    
    if user:
        print("\n====================================")
        print(f"Welcome back, {user['user_name']}!")
        print(f"You are registered for: {user['program']}")
        print(f"Registration ID: {user['registration_id']}")
        print(f"Duration: {program_durations[user['program']]} weeks")
        print("====================================")
    else:
        print("Email not found. Please register first.")

def show_analytics():
    """Display data analytics and visualizations"""
    if not students:
        print("\nNo students to analyze.")
        return
    
    if not ANALYTICS_AVAILABLE:
        print("\nAnalytics visualizations are not available (matplotlib/seaborn not installed).")
        print("Showing text-based analytics only:\n")
    
    try:
        df_analytics = pd.DataFrame(students)
        
        # Program Popularity
        program_counts = df_analytics['program'].value_counts()
        print("\n--- Program Popularity ---")
        for program, count in program_counts.items():
            print(f"{program}: {count} student(s)")
        
        if ANALYTICS_AVAILABLE:
            plt.figure(figsize=(12, 6))
            sns.barplot(x=program_counts.index, y=program_counts.values, palette="viridis")
            plt.xticks(rotation=45, ha="right")
            plt.title("Program Popularity")
            plt.ylabel("Number of Students")
            plt.xlabel("Programs")
            plt.tight_layout()
            plt.show()
        
        # Age Distribution
        print(f"\n--- Age Statistics ---")
        print(f"Average age: {df_analytics['user_age'].mean():.1f}")
        print(f"Youngest student: {df_analytics['user_age'].min()}")
        print(f"Oldest student: {df_analytics['user_age'].max()}")
        
        if ANALYTICS_AVAILABLE:
            plt.figure(figsize=(8, 5))
            sns.histplot(df_analytics['user_age'], bins=10, kde=True, color='skyblue')
            plt.title("Age Distribution of Students")
            plt.xlabel("Age")
            plt.ylabel("Number of Students")
            plt.show()
        
        # Commitment Statistics
        commitment_counts = df_analytics['commitment'].value_counts()
        print(f"\n--- Commitment Statistics ---")
        print(f"Students committing 20+ hours/week: {commitment_counts.get('yes', 0)}")
        print(f"Students committing <20 hours/week: {commitment_counts.get('no', 0)}")
        
        # Most Common Program
        most_common_program = df_analytics['program'].mode()[0]
        print(f"\nMost popular program: {most_common_program}")
        
    except Exception as e:
        print(f"Error during analytics: {e}")

def train_ml_model():
    """Train a simple ML model for program recommendation"""
    if len(students) < 5:  # Need minimum data for meaningful predictions
        return None
    
    try:
        df_ml = pd.DataFrame(students)
        df_ml['commitment_num'] = df_ml['commitment'].map({'yes': 1, 'no': 0})
        
        # Use age and commitment as features
        X = df_ml[['user_age', 'commitment_num']]
        y = df_ml['program']
        
        clf = DecisionTreeClassifier(random_state=42, max_depth=5)
        clf.fit(X, y)
        return clf
    except Exception as e:
        print(f"Error training model: {e}")
        return None

def save_students():
    """Save students to CSV file"""
    try:
        pd.DataFrame(students).to_csv(FILE_NAME, index=False)
        return True
    except Exception as e:
        print(f"Error saving data: {e}")
        return False

# ------------------------------
# Initialize Data
# ------------------------------
# Pre-fill CSV if it doesn't exist
if not os.path.exists(FILE_NAME):
    print(f"Creating new database file: {FILE_NAME}")
    first_names = ["Daniel", "Aisha", "Michael", "Blessing", "Samuel", "Fatima", "David", "Esther",
                   "Joshua", "Zainab", "Emmanuel", "Grace", "Ibrahim", "Cynthia", "Nathaniel",
                   "Maryam", "Peter", "Hannah", "Abdul", "Deborah"]
    last_names = ["Okafor", "Bello", "Johnson", "Adeyemi", "Ogunleye", "Musa", "Williams",
                  "Nwoye", "Brown", "Sadiq", "Olatunji", "Thompson", "Lawal", "Okorie", "Smith",
                  "Abdullahi", "Uche", "Collins", "Rahman", "Akinwale"]

    students = []
    for i in range(20):
        first = first_names[i % len(first_names)]
        last = last_names[i % len(last_names)]
        username = f"{first} {last}"
        email = f"{first.lower()}.{last.lower()}{i+1}@aetherinstitute.com"
        age = random.randint(18, 35)
        program = random.choice(list(programs.keys()))
        registration_id = f"AETHER-{program_codes[program]}-{YEAR}-{i+1:03d}"
        student = {
            "registration_id": registration_id,
            "user_name": username,
            "user_email": email,
            "user_age": age,
            "program": program,
            "commitment": random.choice(["yes", "no"]),
            "learning_institute": "Aether Institute"
        }
        students.append(student)
    
    if save_students():
        print(f"Pre-filled database created with {len(students)} students\n")
else:
    # Load existing students
    try:
        df = pd.read_csv(FILE_NAME)
        students = df.to_dict(orient="records")
        print(f"Loaded {len(students)} existing registrations\n")
    except Exception as e:
        print(f"Error loading data: {e}")
        students = []

# ------------------------------
# Main Portal Loop
# ------------------------------
def main():
    while True:
        print("\n===== Aether Institute Portal =====")
        print("1. View Programs")
        print("2. Register")
        print("3. Program Statistics")
        print("4. Search by Registration ID")
        print("5. Login")
        print("6. Analytics")
        print("7. Exit")

        choice = input("\nSelect an option (1-7): ").strip()

        if choice == "1":
            show_programs()

        elif choice == "2":
            if len(students) >= MAX_STUDENTS:
                print(f"\nRegistration is full. Maximum capacity of {MAX_STUDENTS} students reached.")
                continue

            # Get and validate name
            username = input("\nEnter your full name: ").strip()
            if not username or len(username) < 2:
                print("Please enter a valid name.")
                continue

            # Get and validate email
            email = input("Enter your email: ").strip().lower()
            if not validate_email(email):
                print("Please enter a valid email address.")
                continue

            # Check for duplicates
            if is_duplicate(username, email):
                print("This name or email is already registered.")
                continue

            # Get and validate age
            try:
                age = int(input("Enter your age: ").strip())
                if age < MIN_AGE:
                    print(f"Sorry! You must be at least {MIN_AGE} years old to register.")
                    continue
                if age > MAX_AGE:
                    print(f"Please enter a valid age (under {MAX_AGE}).")
                    continue
            except ValueError:
                print("Please enter a valid number for age.")
                continue

            # Get commitment
            commitment_input = input("Can you commit 20+ hours per week? (yes/no): ").strip()
            commitment = normalize_yes_no(commitment_input)
            if commitment is None:
                print("Please answer 'yes' or 'no'.")
                continue

            show_programs()

            # Offer ML-based suggestion
            predicted_program = None
            if len(students) >= 5:  # Only offer if enough data
                use_suggestion = input("\nWould you like a program suggestion based on your profile? (yes/no): ").strip()
                use_suggestion = normalize_yes_no(use_suggestion)
                
                if use_suggestion == 'yes':
                    clf = train_ml_model()
                    if clf:
                        try:
                            user_features = [[age, 1 if commitment == 'yes' else 0]]
                            predicted_program = clf.predict(user_features)[0]
                            print(f"\nRecommended Program: {predicted_program}")
                            print(f"Duration: {program_durations[predicted_program]} weeks")
                        except Exception as e:
                            print(f"Could not generate suggestion: {e}")

            # Choose program
            if predicted_program:
                program_choice = input("\nEnter program number (or press Enter to accept recommendation): ").strip()
            else:
                program_choice = input("\nEnter program number: ").strip()

            if program_choice == "" and predicted_program:
                selected_program = predicted_program
            else:
                try:
                    program_choice = int(program_choice)
                    if program_choice < 1 or program_choice > len(programs):
                        print(f"Please select a number between 1 and {len(programs)}.")
                        continue
                    selected_program = list(programs.keys())[program_choice - 1]
                except ValueError:
                    print("Please enter a valid program number.")
                    continue

            # Create registration
            registration_id = f"AETHER-{program_codes[selected_program]}-{YEAR}-{len(students)+1:03d}"
            student = {
                "registration_id": registration_id,
                "user_name": username,
                "user_email": email,
                "user_age": age,
                "program": selected_program,
                "commitment": commitment,
                "learning_institute": "Aether Institute"
            }
            students.append(student)
            
            if save_students():
                print("\n" + "="*50)
                print(" REGISTRATION SUCCESSFUL! ")
                print("="*50)
                print(f"Registration ID: {registration_id}")
                print(f"Program: {selected_program}")
                print(f"Duration: {program_durations[selected_program]} weeks")
                commitment_text = "20+ hours/week" if commitment == "yes" else "Less than 20 hours/week"
                print(f"Commitment: {commitment_text}")
                print("\nPlease save your Registration ID for future reference.")
                print("="*50)
            else:
                students.pop()  # Remove if save failed
                print("Registration failed. Please try again.")

        elif choice == "3":
            show_program_statistics()
            
        elif choice == "4":
            search_by_registration_id()
            
        elif choice == "5":
            login()
            
        elif choice == "6":
            show_analytics()
            
        elif choice == "7":
            print("\nThank you for using Aether Institute Portal. Goodbye!")
            break
            
        else:
            print("Invalid option. Please choose between 1 and 7.")

if __name__ == "__main__":
    main()

Creating new database file: students.csv
Pre-filled database created with 20 students


===== Aether Institute Portal =====
1. View Programs
2. Register
3. Program Statistics
4. Search by Registration ID
5. Login
6. Analytics
7. Exit

Available Programs:

1. Data Science
   → Learn data analysis, visualization, and machine learning using Python. (Duration: 8 weeks)

2. Backend Development
   → Build server-side applications and APIs. (Duration: 6 weeks)

3. Frontend Development
   → Create responsive and interactive websites. (Duration: 6 weeks)

4. Cybersecurity
   → Learn how to protect systems and networks. (Duration: 8 weeks)

5. UI/UX Design
   → Design user-friendly digital experiences. (Duration: 4 weeks)

6. Artificial Intelligence
   → Understand intelligent systems and AI models. (Duration: 10 weeks)

7. Cloud Computing
   → Learn cloud infrastructure and deployment. (Duration: 6 weeks)

8. Software Development
   → Learn the full software development lifecycle. (Duration: 12

c:\Users\HP\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(



 REGISTRATION SUCCESSFUL! 
Registration ID: AETHER-CL-2026-021
Program: Cloud Computing
Duration: 6 weeks
Commitment: 20+ hours/week

Please save your Registration ID for future reference.

===== Aether Institute Portal =====
1. View Programs
2. Register
3. Program Statistics
4. Search by Registration ID
5. Login
6. Analytics
7. Exit

Thank you for using Aether Institute Portal. Goodbye!
